In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


C:\Users\ASUS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
def warping(frame,points,w,h,inv=False):
    pts1 = np.float32(points)
    pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])
    if inv:
        matrix =  cv2.getPerspectiveTransform(pts2,pts1) 
    else:
        matrix =  cv2.getPerspectiveTransform(pts1,pts2)
    warpframe = cv2.warpPerspective(frame,matrix,(w,h))
    return warpframe

In [3]:
def blur(frame):
    blur_f = cv2.GaussianBlur(frame, (7,7),0)
    return blur_f
def edge(frame):
    edge_f = cv2.Canny(frame,100,200)
    return edge_f
def hsv_mask_y(frame):
    #hue_min = cv2.getTrackbarPos("hue_min","tracker")
    #sat_min = cv2.getTrackbarPos("sat_min","tracker")
    #value_min = cv2.getTrackbarPos("value_min","tracker")
    #value_max = cv2.getTrackbarPos("value_max","tracker")
    #hue_max = cv2.getTrackbarPos("hue_max","tracker")
    #sat_max = cv2.getTrackbarPos("sat_max","tracker")
    hue_min = 0
    sat_min = 84
    value_min = 160
    value_max = 255
    hue_max = 177
    sat_max = 255
    low = (hue_min,sat_min,value_min)
    high = (hue_max,sat_max, value_max)
    mask= cv2.inRange(frame,low,high)
    return mask
def hsv_mask_w(frame):
    #hue_min = cv2.getTrackbarPos("hue_min","tracker")
    #sat_min = cv2.getTrackbarPos("sat_min","tracker")
    #value_min = cv2.getTrackbarPos("value_min","tracker")
    #value_max = cv2.getTrackbarPos("value_max","tracker")
    #hue_max = cv2.getTrackbarPos("hue_max","tracker")
    #sat_max = cv2.getTrackbarPos("sat_max","tracker")
    #hue_min = 0
    #sat_min = 84
    #value_min = 160
    #value_max = 255
    #hue_max = 177
    #sat_max = 255
    #low = (hue_min,sat_min,value_min)
    #high = (hue_max,sat_max, value_max)
    #mask= cv2.inRange(frame,low,high)
    #min_thres = cv2.getTrackbarPos("min_thres","tracker")
    #max_thres = cv2.getTrackbarPos("max_thres","tracker")
    max_thres=255
    min_thres=211
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    thres = cv2.threshold(gray,min_thres,max_thres,cv2.THRESH_BINARY)[1]
    return thres

def warpingPoints(w=1000,h=700):
    wT = cv2.getTrackbarPos("width top","tracker")
    hT = cv2.getTrackbarPos("hight top","tracker")
    wB = cv2.getTrackbarPos("width bottom","tracker")
    hB = cv2.getTrackbarPos("hight bottom","tracker")
    points = np.float32([(wT,hT),(w-wT,hT),(wB,hB),(w-wB,hB)])
    return points
def display(frame,points):
    for x in range(4):
        cv2.circle(frame,(int(points[x][0]),int(points[x][1])),15,(0,255,255),-1)
    return frame
def nothing(x):
    pass

In [4]:
def getHistogram(frame,thres=0.1,region=1):
    
    if(region==1):
        hist = np.sum(frame,axis=0)
    else:
        hist = np.sum(frame[frame.shape[0]//region:,:],axis=0)
    maxValue = np.max(hist)
    
    minValue = thres*maxValue
    
    indexArray = np.where(hist>=minValue)
    
    basepoint = int(np.average(indexArray))
    
    imgHist = np.zeros((frame.shape[0],frame.shape[1],3),np.uint8)
    for x,intensity in enumerate(hist):
        cv2.line(imgHist,(x,frame.shape[0]),(x,frame.shape[0]-intensity//255//region),(255,255,0),1)
        cv2.circle(imgHist,(basepoint,frame.shape[0]),20,(0,255,0),-1)
    return basepoint,imgHist
    

In [41]:
key = 0
currlistYellow =[]
avgVal = 10
path = "5.mp4"
#cv2.namedWindow("tracker")
w=1000
h=700
#cv2.createTrackbar("hue_min","tracker",0,179,nothing)
#cv2.createTrackbar("hue_max","tracker",94,179,nothing)
#cv2.createTrackbar("sat_min","tracker",39,255,nothing)
#cv2.createTrackbar("sat_max","tracker",255,255,nothing)
#cv2.createTrackbar("value_min","tracker",164,255,nothing)
#cv2.createTrackbar("value_max","tracker",239,255,nothing)
#cv2.createTrackbar("min_thres","tracker",0,255,nothing)
#cv2.createTrackbar("max_thres","tracker",255,255,nothing)
############### points #############

#cv2.createTrackbar("width top","tracker",407,w//2,nothing)
#cv2.createTrackbar("hight top","tracker",434,h,nothing)
#cv2.createTrackbar("width bottom","tracker",145,w//2,nothing)
#cv2.createTrackbar("hight bottom","tracker",613,h,nothing)
wT = 407
hT = 434
wB = 145
hB = 613
cap = cv2.VideoCapture(path)
ret,frame = cap.read()
mask_video = cv2.VideoWriter('mask.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, (w,h))
output_video = cv2.VideoWriter('output.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, (w,h))
while(1):
    ret,frame = cap.read()
    #if(not ret):
        #cap = cv2.VideoCapture(path)
        #continue
    frame=cv2.resize(frame,(w,h))
    h,w,c= frame.shape
    #points = warpingPoints()
    points = np.float32([(wT,hT),(w-wT,hT),(wB,hB),(w-wB,hB)])
    warp = warping(frame,points,w,h)
    hsv = cv2.cvtColor(warp.copy(),cv2.COLOR_BGR2HSV)
    blur_f = blur(hsv)
    mask_yellow = hsv_mask_y(blur_f)
    mask_white = hsv_mask_w(warp.copy())
    mask_f_white = cv2.bitwise_and(warp,warp,mask = mask_white)
    mask_f_yellow = cv2.bitwise_and(warp,warp,mask = mask_yellow)
    mask = mask_yellow | mask_white
    imgRes = np.zeros_like(frame)
    invwarp = warping(mask,points,w,h,inv=True)
    imgRes[:,:,:] = 255,0,0
    imgRes = cv2.bitwise_and(imgRes,imgRes,mask= invwarp)
    img = cv2.bitwise_not(invwarp)
    img=cv2.bitwise_and(frame,frame,mask = img)
    output = img | imgRes
    #imgRes = cv2.bitwise_or(img,img,mask = imgRes)
    #frame=display(frame,points)
    #midPoint,imgM = getHistogram(mask_yellow,thres=0.5,region=4)
    #basePoint,_ = getHistogram(mask_yellow,thres=0.2,region=1)
    #currlistYellow.append(basePoint-midPoint)
    #if(len(currlistYellow)>avgVal):
        #currlistYellow.pop(0)
    #curve = int(sum(currlistYellow)//len(currlistYellow))
    #frameInvWarp = warping(warp,points,w,h,inv=True)
    #frameInvWarp[0:h//3,0:w] = 0,0,0
    #imgLane = np.zeros_like(frame)
    #imgLane[:,:] = 0,255,0
    #imgLane = cv2.bitwise_and(frameInvWarp,imgLane)
    #imgResult = cv2.addWeighted(frame,1,imgLane,1,0)
    key = cv2.waitKey(10)
    cv2.imshow("video", output)
    cv2.imshow("warp",mask)
    output_video.write(output)
    mask_video.write(mask)
    if(key == 27):
        break
cv2.destroyAllWindows()
cap.release()
mask_video.release()
output_video.release()